
# 地域ベースの CNN (R-CNN)

 :ラベル: `sec_rcnn`

 :numref: `sec_ssd`で説明されているシングル ショット マルチボックス検出のほかに、領域ベースの CNN または CNN 特徴を備えた領域 (R-CNN) も、深層学習を物体検出に適用する多くの先駆的なアプローチの 1 つです (引用: `Girshick.Donahue.Darrell.ea.2014` 。このセクションでは、R-CNN とその一連の改良点、つまり高速 R-CNN :cite: `Girshick.2015` 、高速 R-CNN :cite: `Ren.He.Girshick.ea.2015` 、およびマスクを紹介します。 R-CNN :cite: `He.Gkioxari.Dollar.ea.2017` 。スペースが限られているため、ここではこれらのモデルのデザインのみに焦点を当てます。

##  R-CNN

 *Ｒ－ＣＮＮは*、まず入力画像から多く（例えば２０００）*の領域提案*を抽出し（例えば、アンカーボックスも領域提案とみなすことができる）、それらのクラスと境界ボックス（例えばオフセット）にラベルを付ける。 :cite: `Girshick.Donahue.Darrell.ea.2014`次に、CNN を使用して各領域提案に対して順伝播を実行し、その特徴を抽出します。次に、各領域提案の特徴を使用して、この領域提案のクラスと境界ボックスを予測します。

![](../img/r-cnn.svg) :ラベル: `fig_r-cnn`

 :numref: `fig_r-cnn` R-CNN モデルを示します。より具体的には、R-CNN は次の 4 つのステップで構成されます。
1. *選択的検索*を実行して、入力画像上で複数の高品質領域の提案を抽出します (cite: `Uijlings.Van-De-Sande.Gevers.ea.2013` )。これらの提案された領域は、通常、形状やサイズが異なる複数のスケールで選択されます。各領域の提案には、クラスとグラウンド トゥルース境界ボックスがラベル付けされます。
1. 事前トレーニングされた CNN を選択し、出力層の前で切り捨てます。各領域提案のサイズをネットワークが必要とする入力サイズに変更し、順伝播を通じて領域提案に対して抽出された特徴を出力します。
1. 例として、各地域提案の抽出された特徴とラベル付きクラスを取り上げます。オブジェクトを分類するために複数のサポート ベクター マシンをトレーニングします。各サポート ベクター マシンは、サンプルに特定のクラスが含まれているかどうかを個別に判断します。
1. 例として、各領域提案の抽出されたフィーチャとラベル付き境界ボックスを取り上げます。線形回帰モデルをトレーニングして、グラウンド トゥルース境界ボックスを予測します。

 R-CNN モデルは事前学習済み CNN を使用して画像特徴を効果的に抽出しますが、速度が遅くなります。単一の入力画像から数千の領域提案を選択すると想像してください。これには、オブジェクト検出を実行するために数千の CNN 順伝播が必要です。この膨大なコンピューティング負荷により、現実世界のアプリケーションで R-CNN を広く使用することが不可能になります。

## 高速 R-CNN

 R-CNN の主なパフォーマンスのボトルネックは、計算を共有しない、各領域提案に対する独立した CNN 順伝播にあります。これらの領域には通常重複があるため、独立した特徴抽出では計算が何度も繰り返されることになります。 *R-CNN からの高速 R-* CNN の主な改良点の 1 つは、CNN 順伝播が画像全体に対してのみ実行されることです (引用: `Girshick.2015` )。

![](../img/fast-rcnn.svg) :label: `fig_fast_r-cnn`

 :numref: `fig_fast_r-cnn`高速 R-CNN モデルを記述します。その主な計算は次のとおりです。
1.  R-CNN と比較すると、高速 R-CNN では、特徴抽出のための CNN の入力は、個々の領域の提案ではなく画像全体です。さらに、この CNN はトレーニング可能です。入力画像が与えられた場合、CNN 出力の形状を $1 \times c \times h_1 \times w_1$ とします。
1. 選択的検索により $n$ の地域提案が生成されるとします。これらの領域提案 (さまざまな形状) は、CNN 出力上の対象領域 (さまざまな形状) をマークします。次に、これらの関心領域は、簡単に連結できるように、同じ形状の特徴 (たとえば、高さ $h_2$ と幅 $w_2$ が指定されている) をさらに抽出します。これを達成するために、高速 R-CNN は*関心領域 (RoI) プーリング*層を導入します。CNN 出力と領域提案がこの層に入力され、形状 $n \times c \times h_2 \times w_2$ の連結された特徴が出力されます。すべての地域提案についてさらに抽出されます。
1. 全結合層を使用して、連結されたフィーチャを形状 $n \times d$ の出力に変換します。ここで、$d$ はモデル設計によって異なります。
1. 各 $n$ 領域提案のクラスと境界ボックスを予測します。より具体的には、クラス予測と境界ボックス予測で、全結合層の出力をそれぞれ形状 $n \times q$ ($q$ はクラスの数) の出力と形状 $n \times 4$ の出力に変換します。 。クラス予測にはソフトマックス回帰が使用されます。

高速 R-CNN で提案されている関心領域プーリング層は、 :numref: `sec_pooling`で導入されたプーリング層とは異なります。プーリング層では、プーリング ウィンドウ、パディング、ストライドのサイズを指定することで、出力形状を間接的に制御します。対照的に、関心領域プーリング層では出力形状を直接指定できます。

たとえば、各領域の出力の高さと幅をそれぞれ $h_2$ と $w_2$ として指定してみましょう。 $h \times w$ の形状の関心領域ウィンドウの場合、このウィンドウは $h_2 \times w_2$ グリッドのサブウィンドウに分割されます。各サブウィンドウの形状はおよそ $(h/h_2) \times (w/ w_2)$。実際には、サブウィンドウの高さと幅は切り上げられ、最大の要素がサブウィンドウの出力として使用されます。したがって、関心領域プーリング層は、関心領域の形状が異なる場合でも、同じ形状の特徴を抽出することができます。

例として、 :numref: `fig_roi`では、左上の $3\times 3$ の関心領域が $4 \times 4$ 入力で選択されます。この関心領域については、$2\times 2$ の関心領域プーリング層を使用して、$2\times 2$ の出力を取得します。 4 つの分割されたサブウィンドウのそれぞれには、要素 0、1、4、および 5 (5 が最大) が含まれていることに注意してください。 2 と 6 (6 が最大値)。 8 と 9 (9 が最大値)。そして10。 

![](http://d2l.ai/_images/roi.svg) :ラベル: `fig_roi`

以下では、関心領域プーリング層の計算を示します。 CNN で抽出された特徴`X`の高さと幅が両方とも 4 で、チャネルが 1 つしかないとします。


In [ ]:
import torch
import torchvision

X = torch.arange(16.).reshape(1, 1, 4, 4)
X


さらに、入力画像の高さと幅が両方とも 40 ピクセルであり、選択的検索によってこの画像に対して 2 つの領域提案が生成されると仮定します。各領域の提案は 5 つの要素で表されます。オブジェクト クラスと、その左上隅と右下隅の $(x, y)$ 座標が続きます。


In [2]:
rois = torch.Tensor([[0, 0, 0, 20, 20], [0, 0, 10, 30, 30]])


`X`の高さと幅は入力イメージの高さと幅の $1/10$ であるため、指定された`spatial_scale`引数に従って、提案された 2 つの領域の座標に 0.1 が乗算されます。次に、2 つの対象領域が`X`上でそれぞれ`X[:, :, 0:3, 0:3]`および`X[:, :, 1:4, 0:4]`としてマークされます。最後に、$2\times 2$ の関心領域プーリングでは、各関心領域がサブウィンドウのグリッドに分割され、同じ形状 $2\times 2$ の特徴がさらに抽出されます。


In [3]:
torchvision.ops.roi_pool(X, rois, output_size=(2, 2), spatial_scale=0.1)

tensor([[[[ 5.,  6.],
          [ 9., 10.]]],


        [[[ 9., 11.],
          [13., 15.]]]])


## より高速な R-CNN

オブジェクト検出をより正​​確にするために、高速 R-CNN モデルは通常、選択的検索で多数の領域提案を生成する必要があります。精度を損なうことなく領域提案を減らすために、*より高速な R-CNN は、*選択的検索を*領域提案ネットワーク*に置き換えることを提案しています:cite: `Ren.He.Girshick.ea.2015` 。

![](../img/faster-rcnn.svg) :label: `fig_faster_r-cnn`

 :numref: `fig_faster_r-cnn`より高速な R-CNN モデルを示します。高速 R-CNN と比較して、高速 R-CNN は領域提案方法を選択検索から領域提案ネットワークに変更するだけです。モデルの残りの部分は変更されません。地域提案ネットワークは次の手順で機能します。
1. パディング 1 の $3\times 3$ 畳み込み層を使用して、CNN 出力を $c$ チャネルを持つ新しい出力に変換します。このようにして、CNN で抽出された特徴マップの空間次元に沿った各ユニットは、長さ $c$ の新しい特徴ベクトルを取得します。
1. 特徴マップの各ピクセルを中心として、異なるスケールとアスペクト比の複数のアンカー ボックスを生成し、それらにラベルを付けます。
1. 各アンカー ボックスの中心にある長さ $c$ 特徴ベクトルを使用して、このアンカー ボックスのバイナリ クラス (背景またはオブジェクト) と境界ボックスを予測します。
1. 予測クラスがオブジェクトである予測境界ボックスを考えてみましょう。非最大抑制を使用して重複した結果を削除します。オブジェクトの残りの予測境界ボックスは、関心領域プーリング層に必要な領域提案です。

より高速な R-CNN モデルの一部として、領域提案ネットワークがモデルの残りの部分と共同でトレーニングされることは注目に値します。言い換えれば、より高速な R-CNN の目的関数には、オブジェクト検出におけるクラスとバウンディング ボックスの予測だけでなく、領域提案ネットワークにおけるアンカー ボックスのバイナリ クラスとバウンディング ボックスの予測も含まれます。エンドツーエンドのトレーニングの結果、領域提案ネットワークは高品質の領域提案を生成する方法を学習し、データから学習される領域提案の数を減らしてオブジェクト検出の精度を維持します。

## マスク R-CNN

トレーニング データセットで、オブジェクトのピクセル レベルの位置も画像上でラベル付けされている場合、*マスク R-CNN は*そのような詳細なラベルを効果的に活用して、オブジェクト検出の精度をさらに向上させることができます (引用: `He.Gkioxari.Dollar.ea.2017` )。

![](../img/mask-rcnn.svg) :label: `fig_mask_r-cnn`

 :numref: `fig_mask_r-cnn`に示すように、マスク R-CNN は高速な R-CNN に基づいて変更されます。具体的には、マスク R-CNN は、関心領域プーリング層を*関心領域 (RoI) 配向*層に置き換えます。この関心領域位置合わせレイヤーは、双線形補間を使用して特徴マップ上の空間情報を保存します。これは、ピクセル レベルの予測により適しています。このレイヤーの出力には、すべての関心領域の同じ形状の特徴マップが含まれます。これらは、各関心領域のクラスと境界ボックスだけでなく、追加の完全畳み込みネットワークを通じてオブジェクトのピクセル レベルの位置を予測するためにも使用されます。完全畳み込みネットワークを使用して画像のピクセルレベルのセマンティクスを予測する方法の詳細については、この章の後続のセクションで説明します。

## まとめ
- R-CNN は、入力画像から多くの領域提案を抽出し、CNN を使用して各領域提案に対して順伝播を実行してその特徴を抽出し、これらの特徴を使用してこの領域提案のクラスと境界ボックスを予測します。
-  R-CNN からの高速 R-CNN の主な改良点の 1 つは、CNN 順伝播が画像全体に対してのみ実行されることです。また、関心領域プーリング層も導入されているため、異なる形状の関心領域に対して同じ形状の特徴をさらに抽出することができます。
- 高速 R-CNN は、高速 R-CNN で使用される選択的検索を共同トレーニングされた領域提案ネットワークに置き換えるため、前者は領域提案の数を減らしても物体検出の精度を維持できます。
- マスク R-CNN は、より高速な R-CNN に基づいて、完全な畳み込みネットワークをさらに導入し、ピクセル レベルのラベルを活用して物体検出の精度をさらに向上させます。

## 演習
1. 境界ボックスやクラス確率の予測など、オブジェクト検出を単一の回帰問題として組み立てることはできますか? YOLO モデルの設計を参照できます:cite: `Redmon.Divvala.Girshick.ea.2016` 。
1. シングルショットのマルチボックス検出をこのセクションで紹介する方法と比較してください。それらの主な違いは何ですか? :citet: `Zhao.Zheng.Xu.ea.2019`の図 2 を参照してください。



[ディスカッション](https://discuss.d2l.ai/t/1409)
